In [53]:
source("/home/user/data2/lit/bin/lit_utils.R")
source("/home/user/data3/lit/project/sORFs/sORFs.utils.R")
lib_text()
lib_plot()

##### 起始密码子附近的kozak序列 ##### 
get_window_15_start_codon <- function(df){
  df %>% filter(Strand=="+") ->  all_sep_forward
  data.frame(all_sep_forward$Chr,all_sep_forward$ORF_start-15,all_sep_forward$ORF_start+18,all_sep_forward$ORF_id,".",
             all_sep_forward$Strand) -> window_15_start_codon_f
  df %>% filter(Strand=="-") ->  all_sep_reverse
  data.frame(all_sep_reverse$Chr,all_sep_reverse$ORF_end-18,all_sep_reverse$ORF_end+15,all_sep_reverse$ORF_id,".",
             all_sep_reverse$Strand) -> window_15_start_codon_r
  colnames(window_15_start_codon_r) <- colnames(window_15_start_codon_f)
  rbind(window_15_start_codon_f,window_15_start_codon_r) -> window_15_start_codon
  return(window_15_start_codon)
}
## 方法一
# 输入fasta所在的路径
seqlogo_cus <- function(fasta_path,output_path,keep_start_codon_or_not){
  library(seqLogo)
  library(Biostrings)
  # 读取 FASTA 文件
  seqs <- readDNAStringSet(fasta_path)
  # 直接计算碱基频率矩阵（PFM）
  pfm <- consensusMatrix(seqs)[1:4, ]  # 只保留A/C/G/T
  rownames(pfm) <- c("A", "C", "G", "T")
  # 检查PFM（确保无NA/Inf值）
  print(pfm[, 1:5])  # 查看前5列
  if(!keep_start_codon_or_not){
    pfm[,c(1:15,19:33)] -> pfm
  }
  # 转换为PWM（需确保所有列和为1）
  pwm <- makePWM((pfm) / colSums(pfm))  # 加伪计数避免除零错误
  # 绘制Sequence Logo
  pdf(output_path, width = 7, height = 4)  # 可自定义尺寸
  seqLogo(pwm)
  dev.off()
}
## 方法二
# 读取 FASTA 文件
ggseqlogo_cus <- function(fasta_path,output_path,keep_start_codon_or_not){
  library(ggseqlogo)
  library(Biostrings)
  seqs <- readDNAStringSet(fasta_path)
  ggseqlogo(as.character(seqs)) -> p
  if(!keep_start_codon_or_not){
    sub <- sapply(as.character(seqs), function(s) paste0(substr(s, 1, 15), substr(s, 19, 33)))
    ggseqlogo(sub) -> p
  }
  ggsave(p,filename = output_path, width = 7, height = 4)
}
inte_func <- function(sep,output_path,name,only_cano){
  # 参数一：整理好的蛋白质元信息数据框，需要包括起始密码子的位置以及起始密码子的类型（如果需要用到only_cano）
  # 参数二：输出路径
  # 参数三：文件名前缀
  # 参数四：是否只统计起始密码子，需要包括这一列（输入的值为0，1，2）
  if(only_cano==1){
    get_window_15_start_codon(sep %>% filter(Start_codon=="ATG")) -> window_15_start_codon_sep
  }else if(only_cano==0){
    get_window_15_start_codon(sep %>% filter(Start_codon!="ATG")) -> window_15_start_codon_sep
  }else{
    get_window_15_start_codon(sep) -> window_15_start_codon_sep
  }
  create_path(output_path)
  output_bed <- paste0(output_path,name,".bed")
  output_fa <- paste0(output_path,name,".fa")
  output_pdf <- paste0(output_path,name,".pdf")
  fwrite(window_15_start_codon_sep,output_bed,col.names = F,sep = '\t')
  command <- paste0("bedtools getfasta -s -fi /home/user/data3/lit/project/sORFs/07-Genome/results/custom_fa/custom_ref.fa -bed ",
                    output_bed," > ",output_fa)
  system(command)
  ggseqlogo_cus(output_fa,output_pdf,TRUE)
  ggseqlogo_cus(output_fa,paste0(output_path,name,".nosc.pdf"),FALSE)
  seqlogo_cus(output_fa, paste0(output_path,name,".seqlogo.pdf"),TRUE)
}


In [47]:
orfs_path <- "../../processed/augment_orf_table/augmented.tsv"
fread_c(orfs_path) -> orfs

In [48]:
orfs %>% filter(ORF_length<=150) -> df
df %>% filter(ORF_type=="canonical")  %>% filter(!is.na(ORF_start))-> cano_sep
df %>% filter(ORF_type!="canonical") -> noncano_sep
nrow(cano_sep)
nrow(noncano_sep)

[1] 573

[1] 16396

In [45]:
head(cano_sep)

,ORF_id,ORF_type,Start_codon,Isoform_id,Chr,Strand,ORF_seq,ORF_length,Geneid,Isoform_structural_category,⋯,FL_TPM,N,C,A,Gene_type,Unique_peptide_n,mean_relative_iBAQ,ORF_start,ORF_end,CDS_seq
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<dbl>,<int>,<int>,<chr>
1,PB.1003.3:chr1:+|10|581:117:393|canonical|ATG,canonical,ATG,PB.1003.3,chr1,+,MGLEDEQKMLTESGDPEEEEEEEEELVDPLTTVREQCEQLEKCVKARERLELCDERVSSRSHTEEDCTEELFDFLHARDHCVAHKLFNNLK,91,UQCRH,full-splice_match,⋯,38.816051,1.676097409,78.7148080,40.1954527,protein_coding,10,6.760138e-04,46303766,46316584,ATGGGACTGGAGGACGAGCAAAAGATGCTTACCGAATCCGGAGATCCTGAGGAGgaggaagaggaagaggaggaATTAGTGGATCCCCTAACAACAGTGAGAGAGCAATGCGAGCAGTTGGAGAAATGTGTAAAGGCCCGGGAGCGGCTAGAGCTCTGTGATGAGCGTGTATCCTCTCGATCACATACAGAAGAGGATTGCACGGAGGAGCTCTTTGACTTCTTGCATGCGAGGGACCATTGCGTGGCCCACAAACTCTTTAACAACTTGAAA
2,PB.10614.3:chr12:+|11|664:250:598|canonical|ATG,canonical,ATG,PB.10614.3,chr12,+,MTKKRRNNGRAKKGRGHVQPIRCTNCARCVPKDKAIKKFVIRNIVEAAAVRDISEASVFDAYVLPKLYVKLHYCVSCAIHSKVVRNRSREARKDRTPPPRFRPAGAAPRPPPKPM,115,RPS26,full-splice_match,⋯,1.913467,0.876541091,13.8328328,7.3546870,protein_coding,6,2.373685e-04,56042166,56044154,ATGACAAAGAAAAGAAGGAACAATGGTCGTGCCAAAAAGGGCCGCGGCCACGTGCAGCCTATTCGCTGCACTAACTGTGCCCGATGCGTGCCCAAGGACAAGGCCATTAAGAAATTCGTCATTCGAAACATAGTGGAGGCCGCAGCAGTCAGGGACATTTCTGAAGCGAGCGTCTTCGATGCCTATGTGCTTCCCAAGCTGTATGTGAAGCTACATTACTGTGTGAGTTGTGCAATTCACAGCAAAGTAGTCAGGAATCGATCTCGTGAAGCCCGCAAGGACCGAACACCCCCACCCCGATTTAGACCTGCGGGTGCTGCCCCACGTCCCCCACCAAAGCCCATG
3,PB.10717.10:chr12:-|1|2544:7:403|canonical|ATG,canonical,ATG,PB.10717.10,chr12,-,MSKRYLQKATKGKLLIIIFIVTLWGKVVSSANHHKAHHVKTGTCEVVALHRCCNKNKIEERSQTVKCSCFPGQVAGTTRAAPSCVDASIVEQKWWCHMQPCLEGEECKVLPDRKGWSCSSGNKVKTTRVTH,131,TAFA2,incomplete-splice_match,⋯,1.640115,0.688898858,3.4066624,2.0477806,protein_coding,5,7.975134e-05,61710405,61867425,ATGAGTAAGAGATACTTACAGAAAGCAACAAAAGGAAAACTGCTAATAATAATATTTATTGTAACCTTGTGGGGGAAAGTTGTATCCAGTGCAAACCATCATAAAGCTCACCATGTTAAAACGGGAACTTGTGAGGTGGTGGCACTCCACAGATGCTGTAATAAGAACAAGATAGAAGAACGGTCACAAACAGTCAAGTGCTCCTGCTTCCCTGGGCAGGTGGCAGGCACCACGCGAGCTGCTCCATCATGTGTGGATGCTTCAATAGTGGAACAGAAATGGTGGTGCCATATGCAGCCATGTCTAGAGGGAGAAGAATGTAAAGTTCTTCCGGATCGGAAAGGATGGAGCTGTTCCTCTGGGAATAAAGTCAAAACAACTAGGGTAACCCAT
4,PB.10898.1:chr12:+|2|1498:38:485|canonical|ATG,canonical,ATG,PB.10898.1,chr12,+,MKALIVLGLVLLSVTVQGKVFERCELARTLKRLGMDGYRGISLANWMCLAKWESGYNTRATNYNAGDRSTDYGIFQINSRYWCNDGKTPGAVNACHLSCSALLQDNIADAVACAKRVVRDPQGIRAWVAWRNRCQNRDVRQYVQGCGV,148,LYZ,full-splice_match,⋯,0.410029,0.009222873,0.3070568,0.1581398,protein_coding,5,2.435309e-05,69348408,69353219,ATGAAGGCTCTCATTGTTCTGGGGCTTGTCCTCCTTTCTGTTACGGTCCAGGGCAAGGTCTTTGAAAGGTGTGAGTTGGCCAGAACTCTGAAAAGATTGGGAATGGATGGCTACAGGGGAATCAGCCTAGCAAACTGGATGTGTTTGGCCAAATGGGAGAGTGGTTACAACACACGAGCTACAAACTACAATGCTGGAGACAGAAGCACTGATTATGGGATATTTCAGATCAATAGCCGCTACTGGTGTAATGATGGCAAAACCCCAGGAGCAGTTAATGCCTGTCATTTATCCTGCAGTGCTTTGCTGCAAGATAACATCGCTGATGCTGTAGCTTGTGCAAAGAGGGTTGTCCGTGATCCACAAGGCATTAGAGCATGGGTGGCATGGAGAAATCGTTGTCAAAACAGAGATGTCCGTCAGTATGTTCAAGGTTGTGGAGTG
5,PB.11287.6:chr12:+|6|795:93:354|canonical|ATG,canonical,ATG,PB.11287.6,chr12,+,MSLPLNPKPFLNGLTGKPVMVKLKWGMEYKGYLVSVDGYMNMQLANTEEYIDGALSGHLGEVLIRCNNVLYIRGVEEEEEDGEMRE,86,SNRPF,full-splice_match,⋯,6.560459,1.379478018,6.5970894,3.9882837,protein_coding,20,6.614509e-04,95859073,95866071,ATGAGTTTACCCCTCAATCCCAAACCTTTCCTCAATGGACTAACAGGAAAGCCAGTGATGGTGAAACTTAAGTGGGGAATGGAGTACAAGGGCTATCTGGTATCTGTAGATGGCTACATGAACATGCAGCTTGCAAATACAGAAGAATACATAGATGGAGCTTTGTCTGGACATCTGGGTGAAGTTTTAATAAGGTGTAATAATGTCCTTTATATCAGAGGTGTGGAAGAAGAGGAAGAAGATGGGGAAATGAGAGAA
6,PB.11412.30:chr12:-|8|1144:151:367|canonical|ATG,canonical,ATG,PB.11412.30,chr12,-,MPAGVPMSTYLKMFAASLLAMCAGAEVVHRYYRPDLTIPEIPPKRGELKTELLGLKERKHKPQVSQQEELK,71,C12orf73,full-splice_match,⋯,3.963611,1.481402907,5.1283157,3.3048593,protein_coding,2,5.064325e-05,103951522,103956738,ATGCCCGCGGGCGTGCCCATGTCCACCTACCTGAAAATGTTCGCAGCCAGTCTCCTGGCCATGTGCGCAGGGGCAGAAGTGGTGCACAGGTACTACCGAC

In [54]:
nrow(cano_sep)
for (i in 2){
if(i==0){
    prefix="noncano"
}else if(i==1){
    prefix="cano"
}else{
    prefix="all"
}
inte_func(cano_sep,
        paste0("../../figures/cano_sep_scodon_kozak/",prefix,"/"),
        "prefix",
        i
)
}

[1] 573

  [,1] [,2] [,3] [,4] [,5]
A   86   84  109   86  115
C  162  211  186  185  207
G  182  156  149  175  143
T  143  122  129  127  108


In [55]:
nrow(noncano_sep)
for (i in 0:2){
if(i==0){
    prefix="noncano"
}else if(i==1){
    prefix="cano"
}else{
    prefix="all"
}
inte_func(noncano_sep,
        paste0("../../figures/noncano_sep_ms_suppo_scodon_kozak/",prefix,"/"),
        "prefix",
        i
)
}

[1] 16396

  [,1] [,2] [,3] [,4] [,5]
A 2033 2283 2492 2052 2240
C 2282 2430 2414 2389 2472
G 2630 2900 2273 2505 2927
T 2487 1819 2253 2486 1793
  [,1] [,2] [,3] [,4] [,5]
A 1768 1733 1863 1752 1705
C 1569 1653 1618 1546 1648
G 1554 1772 1795 1619 1844
T 2073 1806 1688 2047 1767
  [,1] [,2] [,3] [,4] [,5]
A 3801 4016 4355 3804 3945
C 3851 4083 4032 3935 4120
G 4184 4672 4068 4124 4771
T 4560 3625 3941 4533 3560


In [56]:
# noncano_sep %>% %>% filter(Run_occurrence>=2|Unique_peptide_n>=2)  -> noncano_filtered_sep
noncano_sep %>% filter(Unique_peptide_n>=2)  -> noncano_filtered_sep
nrow(noncano_filtered_sep)
for (i in 0:2){
if(i==0){
    prefix="noncano"
}else if(i==1){
    prefix="cano"
}else{
    prefix="all"
}
inte_func(noncano_filtered_sep,
        paste0("../../figures/noncano_sep_ms_suppo_filtered_scodon_kozak/",prefix,"/"),
        "prefix",
        i
)
}

[1] 611

  [,1] [,2] [,3] [,4] [,5]
A   57   79   77   70   78
C   75   89   98   90   90
G  102  105   83   92  108
T   83   44   59   65   41
  [,1] [,2] [,3] [,4] [,5]
A   69   73   82   53   71
C   67   81   73   73   77
G   85   83   75   77   79
T   73   57   64   91   67
  [,1] [,2] [,3] [,4] [,5]
A  126  152  159  123  149
C  142  170  171  163  167
G  187  188  158  169  187
T  156  101  123  156  108


In [57]:
noncano_sep %>% filter(Unique_peptide_n_msfragger_closed!=0)-> noncano_filtered_sep_based_on_source
nrow(noncano_filtered_sep_based_on_source)
for (i in 0:2){
if(i==0){
    prefix="noncano"
}else if(i==1){
    prefix="cano"
}else{
    prefix="all"
}
inte_func(noncano_filtered_sep_based_on_source,
        paste0("../../figures/noncano_filtered_sep_based_on_source_scodon_kozak/",prefix,"/"),
        "prefix",
        i
)
}

[1] 7051

  [,1] [,2] [,3] [,4] [,5]
A  908  987 1079  882 1004
C  993 1059 1023 1037 1068
G 1102 1231  983 1050 1252
T 1082  808 1000 1116  761
  [,1] [,2] [,3] [,4] [,5]
A  736  727  788  740  730
C  640  708  691  675  701
G  690  759  775  688  789
T  900  772  712  863  746
  [,1] [,2] [,3] [,4] [,5]
A 1644 1714 1867 1622 1734
C 1633 1767 1714 1712 1769
G 1792 1990 1758 1738 2041
T 1982 1580 1712 1979 1507


In [62]:
noncano_sep %>% filter(ORF_type!="noncoding")-> noncano_filtered_sep_based_on_orf_type
nrow(noncano_filtered_sep_based_on_orf_type)
for (i in 0:2){
if(i==0){
    prefix="noncano"
}else if(i==1){
    prefix="cano"
}else{
    prefix="all"
}
inte_func(noncano_filtered_sep_based_on_orf_type,
        paste0("../../figures/noncano_filtered_sep_based_on_orf_type_scodon_kozak/",prefix,"/"),
        "prefix",
        i
)
}

[1] 9274

  [,1] [,2] [,3] [,4] [,5]
A 1181 1279 1431 1188 1285
C 1347 1453 1429 1421 1462
G 1569 1714 1349 1468 1758
T 1392 1043 1280 1412  984
  [,1] [,2] [,3] [,4] [,5]
A  983  951 1017  942  894
C  825  864  857  839  907
G  862 1003  995  900 1023
T 1115  967  916 1104  961
  [,1] [,2] [,3] [,4] [,5]
A 2164 2230 2448 2130 2179
C 2172 2317 2286 2260 2369
G 2431 2717 2344 2368 2781
T 2507 2010 2196 2516 1945


In [61]:
noncano_sep %>% filter(Run_occurrence>1)-> noncano_filtered_sep_based_on_run_occ
nrow(noncano_filtered_sep_based_on_run_occ)
for (i in 0:2){
if(i==0){
    prefix="noncano"
}else if(i==1){
    prefix="cano"
}else{
    prefix="all"
}
inte_func(noncano_filtered_sep_based_on_run_occ,
        paste0("../../figures/noncano_filtered_sep_based_on_run_occ_scodon_kozak/",prefix,"/"),
        "prefix",
        i
)
}

[1] 2959

  [,1] [,2] [,3] [,4] [,5]
A  349  420  435  384  413
C  414  420  430  408  432
G  478  506  423  445  497
T  412  307  365  416  311
  [,1] [,2] [,3] [,4] [,5]
A  319  323  339  320  316
C  317  334  310  284  328
G  288  337  351  314  332
T  382  312  306  388  330
  [,1] [,2] [,3] [,4] [,5]
A  668  743  774  704  729
C  731  754  740  692  760
G  766  843  774  759  829
T  794  619  671  804  641


In [59]:
colnames(noncano_sep)

[1] "ORF_id"                            "ORF_type"                         
 [3] "Start_codon"                       "Isoform_id"                       
 [5] "Chr"                               "Strand"                           
 [7] "ORF_seq"                           "ORF_length"                       
 [9] "Geneid"                            "Isoform_structural_category"      
[11] "Isoform_subcategory"               "Is_uniprot"                       
[13] "Is_canonical"                      "Run_occurrence"                   
[15] "RPF_reads"                         "Psites_number"                    
[17] "RPF_RPKM"                          "Psites_RPKM"                      
[19] "RPF_codon_coverage"                "Psites_codon_coverage"            
[21] "FL"                                "FL_TPM"                           
[23] "N"                                 "C"                                
[25] "A"                                 "Gene_type"                        
[27] "Unique_peptide_n"                  "Unique_peptide_n_msfragger_closed"
[29] "mean_relative_iBAQ"                "ORF_start"                        
[31] "ORF_end"                           "CDS_seq"